In [12]:
import time
import re
import pytz
from pytz import timezone
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string
from nltk import trigrams
import json
import pandas as pd
from unicodedata import normalize
 
punctuation = list(string.punctuation)
city_specific_stop_words =  ['York', 'Manhattan','NY','NewYork','NYC','The','A','#NewYork','#NYC','Brooklyn']
stop = stopwords.words('english') + punctuation + city_specific_stop_words + ['RT', 'rt', 'via', 'I' , 'like','\\',',',':','in','…','\ud83c','\ud83d','°','u2026','ud83d','ud83c']

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)


emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
        
    )"""


regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]


tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

count_all = Counter()

#twitter_dataset = pd.read_table('tweets_streamed_dataset.txt', index_col=None)
twitter_dataset = pd.read_table('data/nyc_streamed_dataset.txt', index_col=None)

for tweet in twitter_dataset.tweet_text:
    #tweet = normalize('NFKD', tweet).encode('ascii','ignore').decode('unicode_escape')
    tweet = normalize('NFKD', tweet)
    terms_stop = [term for term in preprocess(tweet) if term not in stop and not term.startswith(('#', '@')) and not emoji_pattern.search(term)]
    #terms_trigram = trigrams(terms_stop)
    #terms_hash = [term for term in preprocess(tweet1) if term.startswith('#')]
    terms_all = [term for term in preprocess(tweet) if term not in stop]
    #count_all.update(terms_trigram)
    #count_all.update(terms_hash)
    count_all.update(terms_all)
    

print(count_all.most_common(200))
#twitter_dataset

[('New', 1784), ('n', 1638), ('#job', 905), ('Incident', 890), ('ude', 798), ('th', 763), ('#Hiring', 755), ('Street', 685), ('0', 640), ('udc', 624), ('#CareerArc', 602), ('Avenue', 585), ('Exit', 549), ("I'm", 549), ('Update', 507), ('Construction', 481), ('4', 472), ('f', 466), ('udf', 456), ('udd', 430), ('latest', 428), ('amp', 425), ('Cleared', 418), ('Station', 383), ('work', 378), ('ufe', 357), ('This', 356), ("We're", 334), ('#hiring', 327), ('Brooklyn', 320), ('opening', 320), ('great', 309), ('8', 299), ('#Job', 284), ('#nyc', 283), ('Just', 278), ('5', 274), ('WB', 263), ('#Jobs', 260), ('EB', 255), ('fit', 250), ('25', 248), ('Both', 248), ('2', 247), ('East', 243), ('Park', 243), ('directions', 240), ('Can', 237), ('anyone', 230), ('NB', 227), ('recommend', 227), ('Click', 222), ('SB', 219), ('1', 218), ('posted', 217), ('Want', 217), ('apply', 216), ('See', 215), ('3', 205), ('u2764', 201), ('photo', 201), ('day', 199), ('e', 194), ('6', 193), ('9', 182), ('#t1d', 179), 